In [1]:
import geopandas as gpd
import pandas as pd
import os
import pluto_parquet as pp
from zips import unzip_to_temp
from shutil import rmtree
from functools import partial
import warnings
from google.cloud import storage
import tempfile
import logging
from logging.config import fileConfig
from utils import cpu_use

In [2]:
log_file='extract_load.log'
log_path=os.path.join('logs', log_file)

loginipath='logs/logging_config.ini'
fileConfig(loginipath, defaults={'logfilename': log_path})
logger = logging.getLogger('sLogger')

In [3]:
df=gpd.read_file("puma/geo_export_7c144fe1-e65e-426c-807c-77c05a1e4a20.shp")
df=pp.transform_gdf(df)

In [8]:
def get_wkt(geom):
    return geom.wkt

In [19]:
c=df.geometry.apply(get_wkt)

In [32]:
df2=df.to_wkt()

In [33]:
# pd.DataFrame(df2)
df2

,objectid,boro,puma2010,pumaname10,comp2010ap,comp2010,comp2011,comp2012,comp2013,comp2014,...,withdrawn,inactive,cenunits10,shape_area,shape_leng,adjunits10,comp2020,totaladj,total,geometry
0,46.0,Queens,4112.0,"Queens CD 12 - Jamaica, Hollis & St. Albans",304.0,722.0,177.0,165.0,111.0,321.0,...,489.0,529.0,75185.0,2.600506e+08,72776.738837,75185.0,917.0,78789.0,78789.0,"POLYGON ((-73.801391833601 40.666754914352, -7..."
1,28.0,Queens,4111.0,Queens CD 09 - Richmond Hill & Woodhaven,36.0,56.0,163.0,115.0,229.0,30.0,...,99.0,305.0,48151.0,1.325259e+08,56320.701871,48151.0,62.0,49094.0,49094.0,"POLYGON ((-73.821873763675 40.711157252843, -7..."
2,50.0,Brooklyn,4011.0,"Brooklyn CD 09 - Crown Heights South, Prospect...",30.0,34.0,59.0,217.0,207.0,36.0,...,493.0,307.0,42246.0,4.762882e+07,34964.985985,42246.0,574.0,46169.0,46169.0,"POLYGON ((-73.928283912739 40.661455316056, -7..."
3,42.0,Manhattan,3810.0,"Manhattan CD 01 & 02 - Battery Park City, Gree...",661.0,692.0,1359.0,413.0,383.0,-24.0,...,576.0,446.0,90261.0,8.102645e+07,122532.137978,90261.0,726.0,99164.0,99164.0,MULTIPOLYGON (((-74.00779261197999 40.74197910...
4,53.0,Brooklyn,4008.0,Brooklyn CD 05 - East New York & Starrett City,385.0,401.0,253.0,262.0,222.0,27.0,...,412.0,317.0,55471.0,1.643335e+08,97563.118856,55471.0,1024.0,59256.0,59256.0,MULTIPOLYGON (((-73.888286720755 40.6467306605...
5,25.0,Queens,4107.0,Queens CD 04 - Elmhurst & South Corona,167.0,248.0,176.0,195.0,69.0,111.0,...,198.0,329.0,46608.0,1.075111e+08,100409.437469,46817.0,215.0,48564.0,48355.0,"POLYGON ((-73.85696496309799 40.764082699987, ..."
6,37.0,Manhattan,3802.0,"Manhattan CD 09 - Hamilton Heights, Manhattanv...",92.0,78.0,97.0,28.0,9.0,165.0,...,392.0,35.0,51212.0,4.689913e+07,37925.539824,51212.0,391.0,52854.0,52854.0,"POLYGON ((-73.939250290079 40.828304631889, -7..."
7,19.0,Brooklyn,4016.0,"Brooklyn CD 15 - Sheepshead Bay, Gerritsen Bea...",86.0,101.0,91.0,45.0,66.0,25.0,...,271.0,112.0,65220.0,1.207127e+08,109479.807967,65220.0,236.0,66582.0,66582.0,"POLYGON ((-73.94405703131 40.612001080916, -73..."
8,47.0,Staten Island,3902.0,Staten Is. CD 02 - New Springville & South Beach,83.0,117.0,101.0,127.0,19.0,113.0,...,45.0,135.0,50814.0,6.755420e+08,153221.580051,50814.0,108.0,52213.0,52213.0,"POLYGON ((-74.196162245439 40.634786339796, -7..."
9,10.0,Brooklyn,4001.0,Brooklyn CD 01 - Greenpoint & Williamsburg,1924.0,2857.0,1182.0,1138.0,1799.0,1408.0,...,2211.0,1390.0,63829.0,1.149145e+08,73891.837703,63829.0,1457.0,82662.0,82662.0,"POLYGON ((-73.954396987308 40.739123036056, -7..."


In [34]:
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'secret.json'

In [35]:
client = storage.Client()
bucket = client.bucket('raw-pluto')

In [36]:
    with tempfile.NamedTemporaryFile() as fp:
        logger.info(f'{cpu_use()} aggregating files: writing aggregated file to tempfile')
#         countries_gdf.to_file("countries.geojson", driver='GeoJSON')
        df2.to_csv(fp)
        data_shape=df2.shape
        del df2
        logger.info(f'{cpu_use()} deleted in memory dataframe')
        blob_name=f'puma_geography.csv'
        logger.info(f'making blob: {blob_name}')
        blob=bucket.blob(blob_name)
        blob.upload_from_filename(fp.name)
        logger.info(f'succesfully loaded df with rows: {data_shape[0]} and columns: {data_shape[1]} to file: {blob_name}')

INFO - cpu usage: 2.3% aggregating files: writing aggregated file to tempfile
INFO - cpu usage: 56.5% deleted in memory dataframe
INFO - making blob: puma_geography.csv
INFO - succesfully loaded df with rows: 55 and columns: 28 to file: puma_geography.csv


In [8]:
from storage_to_bq import load_from_uri

In [9]:
load_from_uri('gs://raw-pluto/puma_geography.cs', 'pluto-panel.raw_pluto.puma_geography' )

BadRequest: 400 Error while reading data, error message: Input file is not in Parquet format.